COMP 215 - LAB 2 (NEO)
----------------
#### Name: Max Rowley
#### Date: Jan. 16th 2023

This lab exercise is mostly a review of strings, tuples, lists, dictionaries, and functions.

**Building on new concepts from lab 1**:
  * `datetime.date` objects represent a calendar date
  * *list comprehension* provides a compact way to represent map and filter algorithms

**New Python Concepts**:
  * *f-string* simplifies string formatting operations

As usual, the first code cell simply imports all the modules we'll be using...

In [1]:
import datetime, json, requests
from pprint import pprint    # Pretty Print - built-in python function to nicely format data structures

We'll answer some questions about [Near Earth Objects](https://cneos.jpl.nasa.gov/)
> using NASA's API:  [https://api.nasa.gov/](https://api.nasa.gov/#NeoWS)

You should register for your own API key, (but may use the DEMO_KEY to get started).

First we need a short tutorial on python dates and [f-strings](https://realpython.com/python-f-strings/)...

In [2]:
today = datetime.date.today()   # get a date object representing today's date
print(today, type(today))
formatted_date = f'Today is: {today}'   # A format string - notice how the variable `today` is formatted into the stringg
print(formatted_date)

2023-01-16 <class 'datetime.date'>
Today is: 2023-01-16


### Make a query

Let's get some data from the NEO database...
Here's a query that gets the observation "feed" for today.
(Note: I hard-coded the date below to lock down the data for the lab - ideally used `today()` so the notebook is always up-to-date.)

In [ ]:
API_KEY = 'DEMO_KEY'  # substitute your API key here

today = '2023-01-09'  #  Future enhancement:  str(datetime.date.today())   # Today's date as a string!
# Use an f-string here to "format" the date and API key varaibles.
url = f'https://api.nasa.gov/neo/rest/v1/feed?start_date={today}&end_date={today}&api_key={API_KEY}'

response = requests.request("GET", url, headers={}, data={})

data = json.loads(response.text)  # recall json.loads for lab 1

# TIP: print(data) to see the whole data structure returned, here we grab just the list of NEO's:
n_results = data['element_count']
neos = data['near_earth_objects'][today]
print(f'{n_results} Near Earth Objects found for {today}')
pprint(neos)

Next we extract just the potentially hazerdous asteroids, using a Comp115-style list accumulator *loop*:

In [4]:
hazards =  []
for item in neos:
  if item['is_potentially_hazardous_asteroid'] is True:
    hazards.append(item)
print(f'{len(hazards)} potentially hazardous asteroids identified.')

3 potentially hazardous asteroids identified.


## Exercise 1

In the code cell below, **re-write the accumulator loop above** as a [list comprehension](https://realpython.com/lessons/list-comprehensions-overview/) that implements a ["filter"](https://youtu.be/hUes6y2b--0)
Notice how this provides a concise way to "filter" items of interest from a larger data set.

In [9]:
hazards = [asteroid for asteroid in neos if asteroid["is_potentially_hazardous_asteroid"]]

## Fetch Complete Data for One Asteroid

Notice that the record for each `neo` is a dictionary with `id` field that uniquely identifies this record in the database.

We can use this `id` to fetch complete orbital and close approach data for the NEO.

For example, this query fetches the complete data set for the first hazardous asteroid...


In [ ]:
id = hazards[0]['id']
url = f'https://api.nasa.gov/neo/rest/v1/neo/{id}/?api_key={API_KEY}'
response = requests.request("GET", url, headers={}, data={})
data = json.loads(response.text)

pprint(data)

Notice that the `miss_distance` field contains the distance (in various units) by which the NEO missed an "orbiting body".

## Exercise 2

In the code cell below, write a python function that takes a list of "close approach data" as a parameter,
and returns a 2-tuple with the (date, miss km) of the closest approach to Earth in the list (where "miss km" is the miss distance in km).

Hints:
* notice the input is a list of dictionaries.  Each dictionary has a 'close_approach_date", "orbiting_body", and 'miss_distance' field.
* we are only interested in the closest approach to "Earth"
* use a loop if that is easier to understand - we will look at more compact algorithms to solve this problem in class.

Add at least one unit test to check your work - note the test data only needs dictionaries with the fields your function actually uses.


In [18]:
def close_approaches(data): #first try, using a loop
  closestdistance = float("inf")
  closestdate = None
  for approach in data["close_approach_data"]:
    if(approach["orbiting_body"] == "Earth" and float(approach["miss_distance"]["kilometers"]) < closestdistance):
      closestdistance = float(approach["miss_distance"]["kilometers"])
      closestdate = approach["close_approach_date"]
  return (closestdate, closestdistance)

testdata = {
    "close_approach_data":[
        {
            "orbiting_body":"Earth",
            "miss_distance":{"kilometers":100},
            "close_approach_date":"A little while ago."
        },
        {
            "orbiting_body":"Earth",
            "miss_distance":{"kilometers":150},
            "close_approach_date":"During tea-time."
        },
        {
            "orbiting_body":"Earth",
            "miss_distance":{"kilometers":20}, #this one should be the chosen one!
            "close_approach_date":"Last week."
        },
        {
            "orbiting_body":"Earth",
            "miss_distance":{"kilometers":500},
            "close_approach_date":"At night."
        },
    ]
}

print(close_approaches(testdata))

('Last week.', 20.0)


## OPTIONAL - Take your skills to the next level...
## Exercise 3

In the code cell below, write a complete program that:
 1. fetches the list of NEO's for this week.
 2. for each NEO, fetch it's complete orbital data and determine its closest approach to Earth
 3. identify which NEO from this week's data makes the closet approach to earth
 4. print a nice message with information about the NEO, which it will approach the Earth, and how close it will come.

Hints:
* you'll need the start and end date - end date is today, see if you can use a [`timedelta`](https://docs.python.org/3/library/datetime.html#timedelta-objects) object to ge the start date (you can do basic "date math" with `timedelta` and `date` objects!)
* you may need to modify the function we wrote in Ex. 2 to return a triple with the NEO's id included;
* lots of opportunity here for more practice with list comprehensions


In [3]:
# your code here

import datetime, json, requests

debug = lambda x:pprint("DEBUG: '" + str(x) + "', " + str(type(x))) #set it to lambda x:None to silence later

end_date = str(datetime.date.today()) #get today's date
start_date = str(datetime.date.today() - datetime.timedelta(days=7))
API_KEY = 'DEMO_KEY' #this works? sweet!

#now, figure out the URL we need to use
url = f'https://api.nasa.gov/neo/rest/v1/feed?start_date={start_date}&end_date={end_date}&api_key={API_KEY}'

response = requests.request("GET", url, headers={}, data={}) #request the information from NASA

data = json.loads(response.text) #parse the JSON string into a database variable. Yes, all of this is copied from the examples so far.

neos = []
for day in data['near_earth_objects']: #select only the NEOs and put them all in one big list to work on
  neos += data['near_earth_objects'][day]

def close_approaches(data): # Defining this for use below
  closestdistance = float("inf")
  closestdate = None
  for approach in data["close_approach_data"]:
    if(approach["orbiting_body"] == "Earth" and float(approach["miss_distance"]["kilometers"]) < closestdistance):
      closestdistance = float(approach["miss_distance"]["kilometers"])
      closestdate = approach["close_approach_date"]
  return (closestdate, closestdistance)

closestneo = (None, float("inf"))
closest_neo_id = None

for neo in neos: #find the one with the closest approach
  neodata = json.loads(requests.request("GET", "https://api.nasa.gov/neo/rest/v1/neo/" + str(neo["id"]) + "/?api_key={API_KEY}", headers={}, data={}).text) #request information about that specific NEO
  if("error" in neodata):
    break #A bit janky, but jupyter doesn't seem to like it when I use exit()
  neoapproach = close_approaches(neodata)
  if(neoapproach[1] < closestneo[1]): #if the current NEO is actually closer than the previously recorded closest approach, update the closest approach
    closestneo = neoapproach
    closest_neo_id = neo["id"] 

if("error" not in neodata): #check if 
  print("The closest NEO ID found was " + str(closest_neo_id) + ", which will be " + str(closestneo) + "km from Earth at " + closestneo[0] + ".")
else:
  print("...Looks like NASA denied the request. Oops!")
  print(neodata["error"]["message"])


...Looks like NASA denied the request. Oops!
An invalid api_key was supplied. Get one at https://api.nasa.gov:443
